## Mermaid Code Generation

In [ ]:
import os 
os.environ['OPENAI_API_KEY'] = "openai_api_key"    # API KEY

from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
chat = ChatOpenAI(temperature=0)

You can also include a specific theme or application in the prompt to generate the desired diagram.

In [ ]:
import os
import csv

output_folder = 'output_mermaid_code_folder'  # Replace with the path to the output folder to save mermaid codes

def chatgpt():
    
    messages = [
        SystemMessage(content="""your task is to generate different graph TD as shown below as an example by for different random texts and names used in different applications in English language and add more data to it.

        graph TD;
            Start[New Project] --> Requirements[Collect & Validate Requirements];
            Start --> DataRetrieval[Retrieve Data];
            Requirements --> Transformation[Transform Data];
            DataRetrieval --> DatabaseQuery[Query Database];
            DataRetrieval --> ExternalDataFetch[Fetch External Data];
            Requirements --> Rejected[Rejected Requirements];
            Transformation --> Integration[Data Integration];
            DatabaseQuery --> RecordsFound[Database Records Found];
            ExternalDataFetch --> DataReceived[Received Data];
            DataReceived --> Storage&Utilization[Store & Utilize Data];
            RecordsFound --> DataAnalysis[Data Analysis];
            Rejected --> Requirements;
            Integration --> FinalOutput[Generate Final Output];
            DataAnalysis --> FinalOutput;
            Storage&Utilization --> FinalOutput;
            FinalOutput --> End[Project Completion];   
    """),
    ]
    response = chat(messages)
    return response.content

i = 0
# Run the function 5000 times and save the output each time
for i in range(5000):
    i += 1
    output_filename = f"kor_simpgraph_{i}.txt"
    output_path = os.path.join(output_folder, output_filename)
    
    # Call the function and get the output content
    output_content = chatgpt()
    
    # Save the output content to a TXT file
    with open(output_path, 'w') as output_file:
        output_file.write(output_content)   
        print(i)

## Block Diagram Image generation

In [ ]:
import os
import base64
import requests
import random
from PIL import Image as PILImage, UnidentifiedImageError
from io import BytesIO
from urllib.parse import quote_plus
import urllib
from PIL import Image, UnidentifiedImageError

# Define the source and destination folders
source_folder = 'output_mermaid_code_folder'           # folder path that contain the mermaid code generated from above step
destination_folder = 'output_image_folder'             # output folder to save the images generated from mermaid code

# List of different Mermaid styles
mermaid_styles = [
    "default",
    "forest",
    "dark",
    "neutral",
    "note"
    # Add more Mermaid styles as needed
]

# Define the nodes you want to apply colors to
node_color_map = {
    "A": "fill:#FF0000",  # Apply the desired color for node A
    "B": "fill:#00FF00",  # Apply the desired color for node B
    "C": "fill:#0000FF",  # Apply the desired color for node C
    "D": "fill:#FFFF00",  # Apply the desired color for node D
    "E": "fill:#FF00FF",  # Apply the desired color for node E
    "F": "fill:#00FFFF"   # Apply the desired color for node F
}

# Initialize a list of available colors
available_colors = [
    "fill:#FF0000",
    "fill:#00FF00",
    "fill:#0000FF",
    "fill:#FFFF00",
    "fill:#FF00FF",
    "fill:#00FFFF"
]

def generate_mermaid_image(content, filename, style="dark"):
    base64_content = base64.b64encode(content.encode("utf-8")).decode("utf-8")
    css_param = quote_plus(style)
    url = f"https://mermaid.ink/img/{base64_content}?custom={css_param}&theme={style}"

    response = requests.get(url)
    image_data = BytesIO(response.content)
    image = PILImage.open(image_data)
    image_filename = os.path.join(destination_folder, f"{filename}.jpg")
    image.save(image_filename, format="JPEG")

i=0
# Iterate through text files in the source folder
for filename in os.listdir(source_folder):
    i += 1
    if filename.endswith('.txt'):
        base_filename = os.path.splitext(filename)[0]
        print(base_filename)
        image_filename = os.path.join(destination_folder, f"{base_filename}.jpg")
        if not os.path.exists(image_filename):
            file_path = os.path.join(source_folder, filename)
            with open(file_path, 'r', encoding='utf-8') as txt_file:
                content = txt_file.read()

            # Add the desired line to the content
            modified_content = f'%%{{init:{{"theme":"dark"}}}}%%\n{content}'

            # Randomly select a Mermaid style from the available styles
            selected_style = random.choice(mermaid_styles)
            
            # Modify the Mermaid content to apply colors to specific nodes
            mermaid_lines = content.splitlines()
            modified_lines = []
            colored_nodes = set()  # Track colored nodes to avoid duplication
            
            for line in mermaid_lines:
                node_name = line.split("[")[0].strip()  # Extract the node name (e.g., "A", "B", etc.)
                modified_lines.append(line)  # Add the original line
                
                # Check if the node name is in the node_color_map and not already colored
                if node_name in node_color_map and node_name not in colored_nodes:
                    color = random.choice(available_colors)  # Randomly pick a color
                    modified_lines.append(f"style {node_name} {color};")
                    colored_nodes.add(node_name)  # Mark node as colored
            
            modified_content = "\n".join(modified_lines)
            print(i)
            
            generate_mermaid_image(modified_content, base_filename, selected_style)

## Block Diagram Text Generation

In [ ]:
import os
import csv

source_folder = 'input_mermaid_codes_folder'  # Replace with the path to the folder containing the mermaid text files
output_folder = 'output_summary_folder'  # Replace with the path to the output folder to save the generated summaries files

def chatgpt(text):
    
    messages = [
        SystemMessage(content=f"""your task is to convert the given mermaid code as shown below into meaningful sentence by capturing every detail information in English language.
        {text}             
    """),
    ]
    response = chat(messages)
    return response.content

i = 0
# Iterate through text files in the source folder
for filename in os.listdir(source_folder):
    i += 1
    if filename.endswith('.txt'):
        file_path = os.path.join(source_folder, filename)
        with open(file_path, 'r', encoding='utf-8') as txt_file:
            content = txt_file.read() 
            output_content = chatgpt(content)  # Pass the extracted content to chatgpt()
        
        # Save the modified content to a new text file in the output folder
        output_path = os.path.join(output_folder, filename)
        with open(output_path, 'w', encoding='utf-8') as output_file:
            output_file.write(output_content)
            print(i)